In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import tensorflow as tf
from typing import Tuple

Now that you've learned about tensors, variables, and automatic differentiation, you're ready to learn how to define a neural network from scratch using lower-level TensorFlow operations. It's important to understand these foundational concepts because they give you the flexibility to customize your neural networks in any way you like.

As a reminder, here's the structure of the neural network we defined using Keras:

现在您已经了解了张量、变量和自动微分，您已经准备好学习如何使用较低级别的 TensorFlow 操作从头开始定义神经网络。 理解这些基本概念很重要，因为它们可以让您灵活地以任何您喜欢的方式自定义您的神经网络。

提醒一下，这是我们使用 Keras 定义的神经网络的结构：

![Basic neural network](./images/1-fashion-nn.png)

And here's the same neural network, this time showing the $W$ and $b$ parameters we want to learn during training: 

这是同一个神经网络，这次显示了我们想要在训练期间学习的 $W$ 和 $b$ 参数：

![Basic neural network with details](./images/2-fashion-nn-with-details.png)

Notice that our neural network is composed of two `Dense` layers, and that the first those contains a `ReLU` ("Rectified Linear Unit") activation function. In Keras, we can build our model by simply initializing the `Dense` objects with the appropriate activation functions. If we don't want to use Keras, we need to understand the operations performed by a `Dense` layer a bit better and replicate them. It turns out that a `Dense` layer is just about the simplest type of layer we can use, so it's not that hard to understand and replicate. A `Dense` layer takes as input a $W$ parameter, a $b$ parameter, and data $x$ that comes from the previous layer, and produces the following output:

请注意，我们的神经网络由两个“Dense”层组成，第一个层包含一个“ReLU”（“Rectified Linear Unit”）激活函数。 在 Keras 中，我们可以通过使用适当的激活函数简单地初始化“Dense”对象来构建我们的模型。 如果我们不想使用 Keras，我们需要更好地理解“密集”层执行的操作并复制它们。 事实证明，“密集”层几乎是我们可以使用的最简单的层类型，因此理解和复制并不难。 `Dense` 层将 $W$ 参数、$b$ 参数和来自上一层的数据 $x$ 作为输入，并产生以下输出：

$$
z = Wx + b
$$

The output of the first `Dense` layer is then passed as input to a `ReLU` non-linear function in the following way:

然后，第一个`Dense`层的输出作为输入传递给`ReLU`非线性函数，方式如下：

$$
a = ReLU(z)
$$

Let's now see how we can represent these concepts using TensorFlow code.

现在让我们看看如何使用 TensorFlow 代码来表示这些概念。

In [2]:
class NeuralNetwork(tf.keras.Model):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    initializer = tf.keras.initializers.GlorotUniform()
    self.W1 = tf.Variable(initializer(shape=(784, 20)))
    self.b1 = tf.Variable(tf.zeros(shape=(20,)))
    self.W2 = tf.Variable(initializer(shape=(20, 10)))
    self.b2 = tf.Variable(tf.zeros(shape=(10,)))

  def call(self, x: tf.Tensor) -> tf.Tensor:
    x = tf.reshape(x, [-1, 784])
    x = tf.matmul(x, self.W1) + self.b1
    x = tf.nn.relu(x)
    x = tf.matmul(x, self.W2) + self.b2
    return x

Notice that here we're explicitly instantiating parameters $W$ and $b$ using TensorFlow `Variables` (rather than `Tensors`) because their values change during training. Notice also how we initialize their values &mdash; the parameters $b$ are typically initialized to zero, and the parameters $W$ are typically initialized to small random values. There are many methods for parameter initialization, and the choice of technique can have a big effect on how well the neural network trains.

Other than that, you can see that the additions, multiplications, and `ReLU` calls we discussed earlier are reflected in the code as you might expect.

请注意，这里我们使用 TensorFlow 的`Variables`（而不是`Tensors`）显式实例化参数 $W$ 和 $b$，因为它们的值在训练期间会发生变化。 还要注意我们如何初始化它们的值—— 参数 $b$ 通常初始化为零，参数 $W$ 通常初始化为小的随机值。 参数初始化有多种方法，技术的选择对神经网络的训练效果有很大影响。

除此之外，您可以看到我们之前讨论的加法、乘法和`ReLU`调用如您所料反映在代码中。